In [19]:
from transformers import pipeline
import torch 
import librosa
from IPython.display import Audio
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer
from huggingface_hub import login
import numpy as np

In [20]:
torch.__version__

'2.7.0+cpu'

In [21]:
# load data and tokenizer
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
d:\machine-learning\huggingface-langchain\audio-to-text\venv\Lib\site-packages\transformers\models\wav2vec2\tokenization_wav2vec2.py:720: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  warnings.warn(
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [31]:
# load audio
load_audio = "./audio/audio_03.mp3"
speech, sampling_rate = librosa.load(load_audio, sr=16000)

In [32]:
speech, sampling_rate

(array([ 9.0949470e-13, -4.5474735e-13,  1.4779289e-12, ...,
        -2.1523025e-05, -3.6718004e-05, -3.0558447e-05],
       shape=(22656,), dtype=float32),
 16000)

In [33]:
Audio(load_audio, autoplay=True)

In [34]:
# tokenize input
input_values = tokenizer(speech, return_tensors="pt", padding="longest").input_values
input_values

tensor([[0.0012, 0.0012, 0.0012,  ..., 0.0009, 0.0007, 0.0008]])

In [35]:
with torch.no_grad():
    logits = model(input_values).logits

In [36]:
logits

tensor([[[ 11.5666, -20.7923, -20.5119,  ...,  -4.4660,  -6.4575,  -6.6606],
         [ 11.5829, -20.7902, -20.5098,  ...,  -4.4572,  -6.4539,  -6.6611],
         [ 12.4709, -22.9001, -22.6621,  ...,  -3.1938,  -5.9451,  -7.5140],
         ...,
         [ 11.8742, -25.3323, -25.1260,  ...,  -9.1466, -10.1671,  -9.1671],
         [ 11.5920, -20.8068, -20.5264,  ...,  -4.4627,  -6.4655,  -6.6652],
         [ 11.5743, -20.8157, -20.5351,  ...,  -4.4822,  -6.4747,  -6.6630]]])

In [37]:
# decode prediction
predict_ids = torch.argmax(logits, dim=-1)
transcription = tokenizer.batch_decode(predict_ids)[0]

print("Transcription:", transcription)

Transcription: I WANT TO BUY VANANA
